## Disreperancy Global IPC aggregation methods
For some dates different results are gained when calculating the numbers on admin0 and admin1 level in different way. Since the numbers are directly reported in the excel sheet on admin0 level we can use those. Simeltaneously we could sum all the numbers reported on admin2 level and this should give the same results. However, they don't match. This notebook does a short exploration of where the differences occur

In [1]:
import pandas as pd

In [3]:
admin_level=0
country="somalia"

In [4]:
df=pd.read_excel("../Data/GlobalIPC/SOM_globalipc_newcolumnnames.xlsx",index_col=0)

In [14]:
df.loc[:,"date"] = pd.to_datetime(df.loc[:,"date"])

In [16]:
df = df[(df["date"].notnull()) & (df[f"ADMIN{admin_level}"].notnull())]

In [25]:
df.head()

,ADMIN0,ADMIN1,ADMIN2,ADMIN2_ID,Analysis Name,date,Country Population,pop_CS,% of total county Pop,Area Phase,...,ML2_2,perc_ML2_2,ML2_3,perc_ML2_3,ML2_4,perc_ML2_4,ML2_5,perc_ML2_5,ML2_3p,perc_ML2_3p
0,Somalia: Acute Food Insecurity August 2020,NaN,NaN,NaN,NaN,2020-09-01,12327530.0,12327530.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Awdal,NaN,NaN,NaN,NaN,2020-09-01,NaN,724573.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Somalia,Awdal,Baki,18976802.0,Acute Food Insecurity August 2020,2020-09-01,NaN,99157.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Somalia,Awdal,Borama,18976825.0,Acute Food Insecurity August 2020,2020-09-01,NaN,453434.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Somalia,Awdal,Lughaye,18976913.0,Acute Food Insecurity August 2020,2020-09-01,NaN,99157.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df_agg = df[df["ADMIN0"].str.lower().str.fullmatch(country.lower())].groupby(["ADMIN0","date"],as_index=False).sum()
df_precalc = df[df["ADMIN0"].str.lower().str.match(f"{country.lower()}:")]
df_adm1agg = df[~df["ADMIN0"].str.lower().str.contains(f"{country.lower()}")].groupby("date",as_index=False).sum()

In [94]:
df_comb=df_agg.merge(df_precalc,on="date",suffixes=("_agg","_prec")).merge(df_adm1agg.rename(columns={"pop_CS":"pop_CS_adm1"}),on='date')

In [96]:
df_comb["pop_CS_diff_aggprec"]=df_comb["pop_CS_agg"]-df_comb["pop_CS_prec"]
df_comb["pop_CS_diff_aggadm1"]=df_comb["pop_CS_agg"]-df_comb["pop_CS_adm1"]
df_comb["pop_CS_diff_precadm1"]=df_comb["pop_CS_prec"]-df_comb["pop_CS_adm1"]

In [97]:
df_comb[["date","pop_CS_agg","pop_CS_prec","pop_CS_adm1","pop_CS_diff_aggprec","pop_CS_diff_aggadm1","pop_CS_diff_precadm1"]]

,date,pop_CS_agg,pop_CS_prec,pop_CS_adm1,pop_CS_diff_aggprec,pop_CS_diff_aggadm1,pop_CS_diff_precadm1
0,2017-01-01,12273659.0,12327530.0,12327529.0,-53871.0,-53870.0,1.0
1,2017-07-01,12109771.0,12327530.0,12327529.0,-217759.0,-217758.0,1.0
2,2018-01-01,12109771.0,12327530.0,12109771.0,-217759.0,0.0,217759.0
3,2018-07-01,12327532.0,12327530.0,12327532.0,2.0,0.0,-2.0
4,2019-01-01,12327532.0,12327530.0,12327533.0,2.0,-1.0,-3.0
5,2019-08-01,12327532.0,12327530.0,12327531.0,2.0,1.0,-1.0
6,2020-01-01,12327530.0,12327530.0,12327530.0,0.0,0.0,0.0
7,2020-09-01,12327530.0,12327530.0,19530536.0,0.0,-7203006.0,-7203006.0


### Conclusions on ADM0 level:
For the three methods there is large disperancy on 2017-01, 2017-07, 2018-01, and 2020-09   
On 2020-09 there is a disreperancy between df_adm1agg and the other two. This is due to wrong summing in the raw data of the Woqooyi Galbeed region   
On 2017-07 the population of Mudug is larger in the admin1 numbers given in the sheet compared to summing the admin2 regions within Mudug   
On 2017-01 and 2018-01 magically extra population is added to the national total that isn't present in the sum of the admin1 and admin2's 

### ADMIN1

In [71]:
df_adm1_precalc = df[~df["ADMIN0"].str.lower().str.contains(f"{country.lower()}")]
df_adm1_precalc=df_adm1_precalc.drop("ADMIN1",axis=1)
df_adm1_precalc=df_adm1_precalc.rename(columns={"ADMIN0":"ADMIN1"})

In [73]:
admin_level=2

In [74]:
df_adm2 = df[(df["date"].notnull()) & (df[f"ADMIN{admin_level}"].notnull())]

In [75]:
df_adm1_agg = df_adm2.groupby(["date", "ADMIN1"], dropna=False, as_index=False).sum()

In [76]:
df_adm1_agg.head()

,date,ADMIN1,ADMIN2_ID,Country Population,pop_CS,% of total county Pop,Area Phase,CS_1,perc_CS_1,CS_2,...,ML2_2,perc_ML2_2,ML2_3,perc_ML2_3,ML2_4,perc_ML2_4,ML2_5,perc_ML2_5,ML2_3p,perc_ML2_3p
0,2017-01-01,Awdal,50773531.0,0.0,673264.0,0.0,9.0,483122.0,254.0,117913.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-01-01,Bakool,50773606.0,0.0,367227.0,0.0,12.0,191157.0,211.0,61817.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-01,Banadir,12693395.0,0.0,1650228.0,0.0,2.0,1138657.0,69.0,511571.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-01,Bari,76160370.0,0.0,730147.0,0.0,14.0,453476.0,351.0,226830.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-01-01,Bay,50773641.0,0.0,792182.0,0.0,12.0,441051.0,222.0,148907.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
df_adm1_precalc.equals(df_adm1_agg)

False

In [78]:
df_adm1_precalc.shape

(144, 54)

In [79]:
df_adm1_agg.shape

(144, 50)

In [83]:
df_adm1_comb=df_adm1_agg.merge(df_adm1_precalc,on=["date","ADMIN1"],suffixes=("_agg","_prec"))

In [88]:
df_adm1_comb["pop_CS_diff"]=df_adm1_comb["pop_CS_agg"]-df_adm1_comb["pop_CS_prec"]

In [91]:
df_adm1_comb[df_adm1_comb["date"]=="2017-07-01"][["ADMIN1","pop_CS_agg","pop_CS_prec","pop_CS_diff"]]

,ADMIN1,pop_CS_agg,pop_CS_prec,pop_CS_diff
18,Awdal,673264.0,673264.0,0.0
19,Bakool,367227.0,367227.0,0.0
20,Banadir,1650228.0,1650228.0,0.0
21,Bari,730147.0,730147.0,0.0
22,Bay,792182.0,792182.0,0.0
23,Galgaduud,569434.0,569434.0,0.0
24,Gedo,508403.0,508403.0,0.0
25,Hiraan,520686.0,520686.0,0.0
26,Juba Dhexe,362921.0,362921.0,0.0
27,Juba Hoose,489307.0,489307.0,0.0


In [98]:
df_adm1_comb[df_adm1_comb["date"]=="2020-09-01"][["ADMIN1","pop_CS_agg","pop_CS_prec","pop_CS_diff"]]

,ADMIN1,pop_CS_agg,pop_CS_prec,pop_CS_diff
126,Awdal,724573.0,724573.0,0.0
127,Bakool,284353.0,284353.0,0.0
128,Banadir,2228463.0,2228463.0,0.0
129,Bari,712934.0,712934.0,0.0
130,Bay,846600.0,846600.0,0.0
131,Galgaduud,427809.0,427809.0,0.0
132,Gedo,430943.0,430943.0,0.0
133,Hiraan,422993.0,422993.0,0.0
134,Juba Dhexe,286538.0,648936.0,-362398.0
135,Juba Hoose,648936.0,911502.0,-262566.0
